In [54]:

import random
import hashlib
import base64
import math

## $\text{Parte I}$

### Funções utilitárias

In [55]:
Key = tuple[int, int]

# def is_prime(n):
#     """Teste de primalidade de Miller-Rabin. Detalhes: https://pt.wikipedia.org/wiki/Teste_de_primalidade_de_Miller%E2%80%93Rabin"""
#     # 1 < a < n
#     a = random.randint(2, n - 1)
#     d = 0
#     # s = max{r ∈ N | 2^r divide n - 1}
#     for r in range(1, n):
#         if (n - 1) % (2 ** r) == 0:
#             # d = (n - 1) / (2^r)
#             d = (n - 1) // (2 ** r)
#         else:
#             break
        
#     # a^d ≡ 1 mod n
#     if pow(a, d, n) == 1:
#         return True
#     else:
#         return False

def is_prime(n, _precision_for_huge_n=16):
    """Teste de primalidade de Miller-Rabin. Detalhes: https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python:_Probably_correct_answers"""
    def is_prime_trial_division(n: int) -> bool:
        '''Testa se dado inteiro n é primo por tentativa de divisões'''
        if n == 2:
            return True
        if n < 2 or n % 2 == 0:
            return False
        for i in range(3, math.ceil(math.sqrt(n)), 2):
            if n % i == 0:
                return False
        return True
    
    _known_primes = [2] + \
        [x for x in range(3, 1000, 2) if is_prime_trial_division(x)]

    def _try_composite(a, d, n, s):
        if pow(a, d, n) == 1:
            return False
        for i in range(s):
            if pow(a, 2**i * d, n) == n-1:
                return False
        return True # n  is definitely composite
 
    if n in _known_primes:
        return True
    if any((n % p) == 0 for p in _known_primes) or n in (0, 1):
        return False
    d, s = n - 1, 0
    while not d % 2:
        d, s = d >> 1, s + 1
    # Returns exact according to http://primes.utm.edu/prove/prove2_3.html
    if n < 1373653: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3))
    if n < 25326001: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5))
    if n < 118670087467: 
        if n == 3215031751: 
            return False
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7))
    if n < 2152302898747: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11))
    if n < 3474749660383: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13))
    if n < 341550071728321: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13, 17))
    # otherwise
    return not any(_try_composite(a, d, n, s) 
                   for a in _known_primes[:_precision_for_huge_n])


def generate_big_prime_pair(b):
    p = random.getrandbits(b)
    q = random.getrandbits(b)

    while not is_prime(p):
        p = random.getrandbits(b)
    
    while not is_prime(q) or p == q:
        q = random.getrandbits(b)

    return (p, q)   

def I2OSP(x: int, l: int):
    """Integer-to-Octet-String, responsável por converter um inteiro em BASE64.
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    
    # conversão para base 256 com tamanho l
    try:
        return x.to_bytes(l, byteorder='big')
    except OverflowError:
        return x.to_bytes((l+7)//8, byteorder='big')
        

def OS2IP(X: bytes):
    """Octet-String-to-Integer, responsável por converter o texto BASE64 em um inteiro
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    return int.from_bytes(X, byteorder='big')

def OS2B64(X: bytes):
    """Converte um octeto em BASE64"""
    return base64.b64encode(X)

def B642OS(X: bytes):
    """Converte um BASE64 em octeto"""
    return base64.b64decode(X)

def I2B64(x: int):
    """Converte um inteiro em BASE64"""
    return base64.b64encode(str(x).encode())

def B64concat(a: bytes, b: bytes):
    """Concatena dois BASE64"""
    _a = base64.b64decode(a)
    _b = base64.b64decode(b)
    return base64.b64encode(_a + _b)

def bitwise_xor(a: bytes, b: bytes):
    """Operação XOR entre dois bytes"""
    r=b""
    
    for i in range(max(len(a), len(b))):
        if i >= len(a):
            r += b[i].to_bytes(1, byteorder='big')
        elif i >= len(b):
            r += a[i].to_bytes(1, byteorder='big')
        else:
            r += (a[i] ^ b[i]).to_bytes(1, byteorder='big')
    
    return r

def key_len(key: Key):
    """Retorna o número de octetos do modulo n da chave"""
    _, n = key
    return math.ceil(n.bit_length() / 8)

### Geração de chaves

In [56]:

def generate_public_key(phi: int):
    # A chave pública e é um número primo tal que 1 < e < φ(n) e mdc(e, φ(n)) = 1
    e = random.randrange(2, phi)
    
    while not is_prime(e):
        e = random.randrange(2, phi)

    return e

def generate_private_key(e: int, phi: int):
    # A chave privada d é um número tal que e*d ≡ 1 mod φ(n)
    d = pow(e, -1, phi)
    
    return d

def generate_keypair() -> tuple[Key, Key]:
    while True:
        try :
            p, q = generate_big_prime_pair(1024)
            
            n = p * q
            phi = (p - 1) * (q - 1)
            
            e = generate_public_key(phi)
            d = generate_private_key(e, phi)
            break
        except:
            continue
    
    return ((e, n), (d, n))

#### RSA

In [57]:
def RSA_encode(key: Key, m: bytes):
    e, n = key
    
    M = OS2IP(m)
    
    return pow(M, e, n)

def RSA_decode(key: Key, c: int):
    d, n = key
    
    return pow(c, d, n)

#### Padding e criptografia e descriptografia
Detalhes: https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.3.

In [58]:

def MGF(Z: bytes, l: int):
    """MGF utilizando o Hash SHA-3"""
    
    # hLen denota o comprimento nos octetos da saída da função hash
    hLen = hashlib.sha3_256().digest_size
    
    if l > 2**32 * hLen:
        raise ValueError("Máscara muito longa.")
    
    T = b""
    
    for i in range(math.ceil(l / hLen)):
        # Converte i em um octeto C de tamanho 4 com a primitiva I2OSP.
        C = I2OSP(i, 4)
        # Concatena o resultado do hash SHA-3 de seed Z e C com T.
        T += hashlib.sha3_256(Z + C).digest()
    
    # A máscara M é a string consistindo dos primeiros l octetos de T.
    return T[:l]

def OAEP_encode(M: bytes, emLen: int, P: bytes = b""):
    """Encode de OAEP utilizando o Hash SHA-3"""

    hLen = hashlib.sha3_256().digest_size
    mLen = len(M)
    
    if mLen > emLen - 2 * hLen - 2:
        raise ValueError("Mensagem muito grande.")
    
    # Geração de uma string de octetos de comprimento (emLen - mLen - 2 * hLen - 2) consistindo de zeros.
    PS = b"\x00" * (emLen - mLen - 2 * hLen - 2)
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Concatenação de pHash, PS, um octeto 0x01 e a mensagem M no bloco de dados DB.
    DB = pHash + PS + b"\x01" + M
    
    seed = random.getrandbits(hLen * 8).to_bytes(hLen, byteorder='big')
    
    # Mascaramento do bloco de dados
    dbMask = MGF(seed, emLen - hLen - 1)
    maskedDB = bitwise_xor(DB, dbMask)

    # Mascaramento da seed
    seedMask = MGF(maskedDB, hLen)
    maskedSeed = bitwise_xor(seed, seedMask)
    
    return b'\x00' + maskedSeed + maskedDB

def OAEP_decode(EM: bytes, P: bytes = b""):
    """Decode de OAEP utilizando o Hash SHA-3"""
    hLen = hashlib.sha3_256().digest_size
    emLen = len(EM)
    
    if emLen < 2 * hLen + 2:
        raise ValueError("Erro na decodificação. Tamanho de EM inválido.")
    
    maskedSeed = EM[1: 1 + hLen]
    maskedDB = EM[1 + hLen:]
    
    seedMask = MGF(maskedDB, hLen)
    seed = bitwise_xor(maskedSeed, seedMask)
    
    dbMask = MGF(seed, emLen - hLen - 1)
    DB = bitwise_xor(maskedDB, dbMask)
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Separa a mensagem supondo o formato 00 || pHash_ || PS || 01 || M
    pHash_ = DB[:hLen]        
    
    if pHash != pHash_:
        raise ValueError("Erro na decodifiocação.")
    
    # Busca o byte 01 após o padding.
    i = hLen
    while DB[i] == 0:
        i += 1
    
    if DB[i] != 1:
        raise ValueError("Byte 0x01 não encontrado.")
    
    return DB[i+1:]


#### Criptografia e descriptografia

In [59]:
def encrypt(key: Key, M64: bytes, P: bytes = b""):
    """"Criptografa a mensagem usando RSA-OAEP"""
    try:
        e, _ = key
        
        M = B642OS(M64)
        
        emLen = key_len(key)
        
        EM = OAEP_encode(M, emLen, P)

        c = RSA_encode(key, EM)
        
        C = I2OSP(c, emLen)
        
        C64 = OS2B64(C)
        
        return C64
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None
    
def decrypt(key: Key,  C64: bytes, P: bytes = b""):
    """"Descriptografa a mensagem usando RSA-OAEP"""
    try:
        emLen = key_len(key)
        
        C = B642OS(C64)
        
        c = OS2IP(C)
        
        m = RSA_decode(key, c)
        
        EM = I2OSP(m, emLen)

        M = OAEP_decode(EM, P)
        
        M64 = OS2B64(M)
        
        return M64
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None

    

#### Teste

In [60]:
public_key, private_key = generate_keypair()

print(f"Chave pública: {I2B64(public_key[0]).decode()}")
print(f"Chave privada: {I2B64(private_key[0]).decode()}")

Chave pública: NDYxMTc5MzM3MzIyMzA0Mzg0NTAzNTI2NDY3NjcyNDMwMTAyNjQ0MTExNDk2ODE4Mzc1MTE5Mjc5MTMxODM5NzM5MDM5NTI2NTI3NjcyOTA5NjM5NzA4NDQyNjAyMTI3MzUxMDUzMDg2ODA5MDc2Njg4OTU0NzgzNTEyOTU2MDYxODk5ODQ0NDUxMzc0OTc4NzQxNTU5MTU4NjA4MTUxODY0MTE3MzgxMjQwNDEzNDQ4MjExNzQ3MjYyNjkzNTIxNTU1OTQ3NDY4MjA0NDc4Mjg5OTE2MTI0NjQ4ODgyNjcyNDQ4MzMzNDIyMTY0OTU0NDE5NDM1MDc3NDEzMTk2MTYzNzYwOTU5ODA3MTI0MjIxMTk3NjgyMTY5OTY5OTUxNTU3NTIzOTY0MjQ3MzgyMzU1NDEwMjM3MjE2NDc4OTAyNjkyMjAzOTUwMDg2NzAxOTk2ODE4MjgxMDQzMDc5OTczNzAwNjg5NTg1ODUzNjM3OTE5NDAyMTQ1ODA5NTQxNDI3NTY4ODI0Njk1NzQzMTMxNTI1ODA0Nzc4MjUwOTk0MDAwODc0Mjc3MzYxMDg4NjA2ODQyNTMzNzk1Mjc4NTM0MzczMjA2ODI1MTY5ODc3NTg3ODM3MjA1NDA2NTgwMDE4NDIzMTY4Mjk5MzA1OTAyMDkwODU1NDU2NTU3MDgyNzY1Nzk1NzY5NjI3NTg0MDkxMDg1Nzk0ODg2MTYxNTM5NjIxMzMyNTQ0Mjc0MDIwNzA5Mjc5OTk2Mzk3MjIyOTkwNzAxNTQwMjk5NjUxNTU1MzczNTg5Njc3ODU2NDYyODg4NTc1OQ==
Chave privada: NjQxMTM3NjAzMjIzOTIyOTc1ODM5MTQ0MzY3ODgwNDg4Mjc2NTQ1MzI2OTY2ODkzNTMxMTY1MjA2NzUxODE5OTY1NDIyMzgwOTYwNjIwNjYxMDE2NDYwODEzNTg3MDYxNzg3OTk1NDIxOTAxM

In [86]:
M = OS2B64(b"Foobar supremacy!")

print(f"Mensagem original: {M}")

C = encrypt(private_key, M)

M_ = decrypt(public_key, C)

print(f"Mensagem cifrada (base64): {OS2B64(C)}")

print(f"Mensagem decifrada: {M_}")

if M == M_:
    print("Sucesso!")
else:
    print("Falha!")

Mensagem original: b'Rm9vYmFyIHN1cHJlbWFjeSE='
Mensagem cifrada (base64): b'S00yWGhUYUFVR2xrUWtWTllzeWtiUkVTYzZ6akl1MTQwUXlwVE4rZURiSVBZZWFpVVBRM05TNW5lalo1ZWZ2VlBmd3NDT21ZajlJb1plTzAzcDNMdWRrOGltK01oWGlaQ3BmNDFSODV6NUFkM2Rxb1FTMVlqZlFNa0poN1Q5ekluelhBMVdTRmRnaldLQjJjUFg4RnpoaWdmU3RLS0t2WjM3MFZCdCtuWlQrMzJJbk9NZlZBbEZycnBBOXg5WnFOa0lIMkFmZTRNdVRlZGFJV0lmVlF4azgySlBwOXJVSXpWVS9UcDQwVFdrdGtNK3RkSHlRSXcxSkdZRWViQWJrbWU0RElrNEJVZ2I2RHlSZnA3NnkzV09nOFR1enJMcSthZWNpSklVdkdRK2VvYUhhdldYTXEreGZrNTdodEp5djczRTVaRXpoSCtYaGR2TmhOckFjQ3lnPT0='
Mensagem decifrada: b'Rm9vYmFyIHN1cHJlbWFjeSE='
Sucesso!


## $\text{Parte II}$
### Assinatura

In [62]:

def sign(key: Key, M: bytes):
    """Assinatura digital usando RSA"""
    try:
        hashedM = OS2B64(hashlib.sha3_256(B642OS(M)).digest())

        encryptedHash = encrypt(key, hashedM)
        
        return B64concat(encryptedHash, M)
    
    except ValueError as e:
        print(f"Erro de assinatura: {e}")
        return None

## $\text{Parte III}$
### Verificação

In [87]:
def verify(key: Key, signedM: bytes):
    """Verificação da assinatura digital usando RSA"""
    try:
        emLen = key_len(key)
        
        OSsignedM = B642OS(signedM)
        
        encryptedHash = OS2B64(OSsignedM[:emLen])

        decryptedHash = decrypt(key, encryptedHash)
        
        hashedM = OS2B64(hashlib.sha3_256(OSsignedM[emLen:]).digest())
        
        return hashedM == decryptedHash
        
    except ValueError as e:
        print(f"Erro de verificação: {e}")
        return False

## Criptografia + Assinatura de arquivo

In [64]:
with open("lorem.txt", "rb") as f:   
    M = base64.b64encode(f.read())
    signedM = sign(private_key, M)

    
    if signedM:
        with open("lorem_signed.txt", "wb") as f:
            f.write(signedM)

In [65]:
with open("lorem_signed.txt", "rb") as f:  
    C = f.read()

    M_ = verify(public_key, C)
        
print(M_)
    


True
